# IA3를 활용한 Koalpaca fine-tuning하기

- IA3 방식을 활용하여 Koalpac를 fine-tuning하는 코드 예제입니다.
- 베이스 라인 코드는 beomi님의 베이스 라인 코드를 활용하였습니다! (https://github.com/Beomi/KoAlpaca) 

## 1.라이브러리 다운로드


- 밑의 "##" 표시 아래의 라이브러리는, PEFT의 라이브러리를 수정 후 제가 올려놓은 라이브러리입니다. 살짝 수정하였습니다 :)

- IA3을 'CASUAL_LM' task에 적용할 수 있도록 수정한 코드입니다..!!

- 모든 저작권은 원 저자들에게 있습니다!

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git

###########################################################################
!pip install -q -U git+https://github.com/dopeornope-Lee/peft_modifier.git
#############################################################################

!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

## 2. Datset preparation
- 위의 data는 KoAlpaca v1.1 데이터 셋이며, beomi님이 huggingface에 공개한 dataset입니다!


In [1]:
from datasets import load_dataset

data = load_dataset("beomi/KoAlpaca-v1.1a")

Found cached dataset parquet (/home/aimv/.cache/huggingface/datasets/beomi___parquet/beomi--KoAlpaca-v1.1a-1465f66eb846fd61/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/1 [00:00<?, ?it/s]

In [2]:
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'url'],
        num_rows: 21155
    })
})

In [3]:
# data preparation 
data = data.map(
    lambda x: {'text': f"{ x['instruction']}\n\n 정답: {x['output']}<|endoftext|>" }
)

Loading cached processed dataset at /home/aimv/.cache/huggingface/datasets/beomi___parquet/beomi--KoAlpaca-v1.1a-1465f66eb846fd61/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-eefe2730154e0e25.arrow


## 3. Model & Tokenizer 불러오기

- K(G)OAT는 KoAlpaca를 QLoRA 방식이 아닌, IA3방식으로 훈련시킨 모델입니다.

- 아쉽게도 IA3sms 4bit를 지원하지 않아 8bit로 불러와야 합니다.

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


# 모델은 KoAlpaca-Polyglot-5.8B 모델을 활용하였습니다
model_id = "beomi/KoAlpaca-Polyglot-5.8B"  
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_use_double_quant=True,
    bnb_8bit_quant_type="nf4",
    bnb_8bit_compute_dtype=torch.bfloat16
)
# 토크나이저는 EleutherAI의 polyglot-ko-5.8b를 사용하였습니다
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/polyglot-ko-5.8b")
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map='auto')


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/aimv/anaconda3/envs/ailab/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA SETUP: WARNING! libcuda.so not found! Do you have a CUDA driver installed? If you are on a cluster, make sure you are on a CUDA machine!
CUDA SETUP: CUDA runtime path found: /home/aimv/anaconda3/envs/ailab/lib/libcudart.so.11.0
CUDA SETUP: Loading binary /home/aimv/anaconda3/envs/ailab/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/aimv/anaconda3/envs/ailab/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: No GPU detected! Check your CUDA paths. Proceeding to load CPU-only library...
  warn(msg)
2023-08-14 15:40:44.802185: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-14 15:40:44.826291: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-14 15:40:45.189004: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

## 텍스트 데이터만 tokenize

In [5]:
# text 데이터만 훈련을 위해 Tokenizer를 통해 토큰화 해줍니다!
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Loading cached processed dataset at /home/aimv/.cache/huggingface/datasets/beomi___parquet/beomi--KoAlpaca-v1.1a-1465f66eb846fd61/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-83a23ee55ab85614.arrow


In [7]:
# data체킹을 해봅시다!
data['train'][0]['text']

' 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?\n\n정답: 양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.'

In [6]:
# 아래의 함수는 beomi님의 코드를 활용하였으며, 현재 훈련에 사용되는 파라미터의 수를 
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [7]:
from peft import get_peft_model,IA3Config



config=IA3Config(task_type="CAUSAL_LM")#(target_modules=["query_key_value"],


model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 802816 || all params: 5885861888 || trainable%: 0.013639735611818691


In [8]:
# GPU 할당이 어떻게 되어있는지 체킹하여 봅니다.
for i in model.named_parameters():
    print(f"{i[0]} -> {i[1].device}")

base_model.model.gpt_neox.embed_in.weight -> cuda:0
base_model.model.gpt_neox.layers.0.input_layernorm.weight -> cuda:0
base_model.model.gpt_neox.layers.0.input_layernorm.bias -> cuda:0
base_model.model.gpt_neox.layers.0.post_attention_layernorm.weight -> cuda:0
base_model.model.gpt_neox.layers.0.post_attention_layernorm.bias -> cuda:0
base_model.model.gpt_neox.layers.0.attention.query_key_value.weight -> cuda:0
base_model.model.gpt_neox.layers.0.attention.query_key_value.bias -> cuda:0
base_model.model.gpt_neox.layers.0.attention.query_key_value.ia3_l.default -> cuda:0
base_model.model.gpt_neox.layers.0.attention.dense.weight -> cuda:0
base_model.model.gpt_neox.layers.0.attention.dense.bias -> cuda:0
base_model.model.gpt_neox.layers.0.mlp.dense_h_to_4h.weight -> cuda:0
base_model.model.gpt_neox.layers.0.mlp.dense_h_to_4h.bias -> cuda:0
base_model.model.gpt_neox.layers.0.mlp.dense_4h_to_h.weight -> cuda:0
base_model.model.gpt_neox.layers.0.mlp.dense_4h_to_h.bias -> cuda:0
base_model.mo

## 4. Trainung
- IA3 Config를 사용하여 모델을 fine tuning 해봅니다!

In [9]:
import transformers

tokenizer.pad_token = tokenizer.eos_token


trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        num_train_epochs=4,
        max_steps=15930, # 가장 최적화된 15930 step까지 훈련되었습니다.
        learning_rate=5e-5,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim='adamw_torch'),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 21300543 (aimv). Use `wandb login --relogin` to force relogin


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,0.579300
20,0.660900
30,0.576400
40,0.654800
50,0.620900
60,0.660600
70,0.578100
80,0.623900
90,0.480800
100,0.442300


TrainOutput(global_step=15930, training_loss=0.41051217065393286, metrics={'train_runtime': 206253.6556, 'train_samples_per_second': 0.154, 'train_steps_per_second': 0.077, 'total_flos': 3.0153567596190106e+17, 'train_loss': 0.41051217065393286, 'epoch': 1.51})

In [ ]:
# Inference시 실행시키는 코드입니다!
# model.eval()
# model.config.use_cache = True  

## 5. Few shot 정량 평가입니다.

- 벤치마크 성능비교를 위한 영화리뷰 데이터셋인 NSMC 데이터를 가져와 Fewshot 평가를 진행합니다
- 해당 코드는 beomi님의 Fewshot 평가 procedure를 그대로 따라가며 평가 성능을 도출합니다.


In [10]:
!mkdir -p data_in/KOR/naver_movie
!wget https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/7.PRETRAIN_METHOD/data_in/KOR/naver_movie/ratings_train.txt \
              -O data_in/KOR/naver_movie/ratings_train.txt
!wget https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/7.PRETRAIN_METHOD/data_in/KOR/naver_movie/ratings_test.txt \
              -O data_in/KOR/naver_movie/ratings_test.txt

--2023-08-17 00:59:05--  https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/7.PRETRAIN_METHOD/data_in/KOR/naver_movie/ratings_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘data_in/KOR/naver_movie/ratings_train.txt’

data_in/KOR/naver_m 100%[===================>]  13.95M  13.7MB/s    in 1.0s    

2023-08-17 00:59:07 (13.7 MB/s) - ‘data_in/KOR/naver_movie/ratings_train.txt’ saved [14628807/14628807]

--2023-08-17 00:59:08--  https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/7.PRETRAIN_METHOD/data_in/KOR/naver_movie/ratings_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.g

In [11]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
#os.environ["CUDA_VISIBLE_DEVICES"]= "0"  # Set the GPUs 2 and 3 to use

from transformers import AutoModelForCausalLM, AutoTokenizer

import torch
import pandas as pd
import numpy as np
import re

import random
from random import sample

from tqdm import tqdm
import time

- 시드 넘버를 통해서, 동일한 시드를 통해 평가할 수 있도록 합니다.

In [12]:
SEED_NUM = 1234
np.random.seed(SEED_NUM)
random.seed(SEED_NUM)

In [13]:
DATA_IN_PATH = './data_in/KOR'
DATA_OUT_PATH = './data_out/KOR'

DATA_TRAIN_PATH = os.path.join(DATA_IN_PATH, 'naver_movie', 'ratings_train.txt')
DATA_TEST_PATH = os.path.join(DATA_IN_PATH, 'naver_movie', 'ratings_test.txt')

train_data = pd.read_csv(DATA_TRAIN_PATH, header = 0, delimiter = '\t', quoting = 3)
train_data = train_data.dropna()

In [14]:
print('데이터 positive 라벨: ', '긍정')
print('데이터 negative 라벨: ', '부정')

데이터 positive 라벨:  긍정
데이터 negative 라벨:  부정


In [15]:
print('gpt 최대 토큰 길이: ', model.config.max_position_embeddings)

gpt 최대 토큰 길이:  2048


In [16]:
sent_lens = [len(tokenizer(s).input_ids) for s in tqdm(train_data['document'])]

print('Few shot 케이스 토큰 평균 길이: ', np.mean(sent_lens))
print('Few shot 케이스 토큰 최대 길이: ', np.max(sent_lens))
print('Few shot 케이스 토큰 길이 표준편차: ',np.std(sent_lens))
print('Few shot 케이스 토큰 길이 80 퍼센타일: ',np.percentile(sent_lens, 80))

100%|██████████| 149995/149995 [00:05<00:00, 26492.12it/s]

Few shot 케이스 토큰 평균 길이:  20.22912763758792
Few shot 케이스 토큰 최대 길이:  280
Few shot 케이스 토큰 길이 표준편차:  16.48828728915166
Few shot 케이스 토큰 길이 80 퍼센타일:  27.0


In [17]:
train_fewshot_data = []

for train_sent, train_label in tqdm(train_data[['document', 'label']].values):
    tokens = tokenizer(train_sent).input_ids

    if len(tokens) <= 25:
        train_fewshot_data.append((train_sent, train_label))

  0%|          | 0/149995 [00:00<?, ?it/s]

100%|██████████| 149995/149995 [00:05<00:00, 25731.77it/s]


In [18]:
test_data = pd.read_csv(DATA_TEST_PATH, header=0, delimiter='\t', quoting=3)
test_data = test_data.dropna()
test_data.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [19]:
# Full Dataset
# sample_size = len(test_data)

# Sampled Dataset
sample_size = 500

train_fewshot_samples = []

for _ in range(sample_size):
    fewshot_examples = sample(train_fewshot_data, 10)
    train_fewshot_samples.append(fewshot_examples)

if sample_size < len(test_data['id']):
    test_data = test_data.sample(sample_size, random_state=SEED_NUM)

In [20]:
def build_prompt_text(sent):
    return "문장: " + sent + '\n감정:'

def clean_text(sent):
    sent_clean = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", sent)
    return sent_clean

time_lst=[]
real_labels = []
pred_tokens = []

total_len = len(test_data[['document','label']].values)

for i, (test_sent, test_label) in tqdm(enumerate(test_data[['document','label']].values), total=total_len):
    prompt_text = ''

    for ex in train_fewshot_samples[i]:
        example_text, example_label = ex
        cleaned_example_text = clean_text(example_text)
        appended_prompt_example_text = build_prompt_text(cleaned_example_text)
        appended_prompt_example_text += ' 긍정\n' if example_label == 1 else ' 부정\n'
        prompt_text += appended_prompt_example_text

    cleaned_sent = clean_text(test_sent)
    appended_prompt_sent = build_prompt_text(cleaned_sent)

    prompt_text += appended_prompt_sent

    tokens = tokenizer(prompt_text, return_tensors="pt")
    token_ids, attn_mask = tokens.input_ids.cuda(), tokens.attention_mask.cuda()

    st_point=time.time()

    gen_tokens = model.generate(input_ids=token_ids, attention_mask=attn_mask,
                                    max_new_tokens=1, pad_token_id=0)
    end_point=time.time()
    pred = tokenizer.batch_decode(gen_tokens[:, -1])[0].strip()

    pred_tokens.append(pred)
    diff= end_point-st_point
    time_lst.append(diff)
    real_labels.append('긍정' if test_label == 1 else '부정')

  0%|          | 0/500 [00:00<?, ?it/s]/home/aimv/anaconda3/envs/ailab/lib/python3.10/site-packages/transformers/generation/utils.py:1264: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
100%|██████████| 500/500 [12:17<00:00,  1.48s/it]


In [21]:
# 추론시간 체킹
print(f"{np.mean(np.array(time_lst)):.5f}")

1.47362


In [22]:
accuracy_match = [p == t for p, t in zip(pred_tokens, real_labels)]
accuracy = len([m for m in accuracy_match if m]) / len(real_labels)

print(f"{accuracy:.4f}")

0.712
0.7120


In [24]:
def build_prompt_text(sent):
    return '다음 문장은 긍정일까요 부정일까요?\n' + sent + '\n정답:'

real_labels = []
pred_tokens = []


real_labels = []
pred_tokens = []

time_lst2=[]

total_len = len(test_data[['document','label']].values)

for i, (test_sent, test_label) in tqdm(enumerate(test_data[['document','label']].values), total=total_len):
    prompt_text = ''

    for ex in train_fewshot_samples[i]:
        example_text, example_label = ex
        cleaned_example_text = clean_text(example_text)
        appended_prompt_example_text = build_prompt_text(cleaned_example_text)
        appended_prompt_example_text += ' 긍정\n' if example_label == 1 else ' 부정\n' 
        prompt_text += appended_prompt_example_text

    cleaned_sent = clean_text(test_sent)
    appended_prompt_sent = build_prompt_text(cleaned_sent)

    prompt_text += appended_prompt_sent

    tokens = tokenizer(prompt_text, return_tensors="pt")
    token_ids, attn_mask = tokens.input_ids.cuda(), tokens.attention_mask.cuda()



    st_point=time.time()




    gen_tokens = model.generate(input_ids=token_ids, attention_mask=attn_mask,
                                    max_new_tokens=1, pad_token_id=0)

    
    end_point=time.time()
    diff= end_point-st_point

    pred = tokenizer.batch_decode(gen_tokens[:, -1])[0].strip()
    time_lst2.append(diff)
    pred_tokens.append(pred)
    real_labels.append('긍정' if test_label == 1 else '부정')

100%|██████████| 500/500 [15:48<00:00,  1.90s/it]


In [25]:
accuracy_match = [p == t for p, t in zip(pred_tokens, real_labels)]
accuracy = len([m for m in accuracy_match if m]) / len(real_labels)

print(f"{accuracy:.4f}")

0.8100


In [26]:
print(f"{np.mean(np.array(time_lst2)):.5f}")

1.89420


## 6. 모델 저장 및 업로드방법

- 모델을 불러오는 코드는 추가적으로 평가지표만 도출하는 코드에서 공유하도록 하겠습니다.

- 모델을 이런방식으로 저장하게 되면, 어탭터만 저장되게 됨으로, model.config를 통하여 config를 확인하여 허깅페이스에 같이 업로드 해주면 좋습니다.

In [ ]:
MODEL_SAVE_REPO = '저장할 모델의 이름을 입력해주세요' # ex) 'KOAT-5.8b'
HUGGINGFACE_AUTH_TOKEN = '본인의 Huggingface 토큰을 넣어주세요' # https://huggingface.co/settings/token
 
## Push to huggingface-hub
model.push_to_hub(
			MODEL_SAVE_REPO, 
			use_temp_dir=True, 
			use_auth_token=HUGGINGFACE_AUTH_TOKEN
)
tokenizer.push_to_hub(
			MODEL_SAVE_REPO, 
			use_temp_dir=True, 
			use_auth_token=HUGGINGFACE_AUTH_TOKEN
)